In [1]:
# Scrape LinkedIn companies based on user inputted search query and update into Company Web Scrape Google Sheets 
# Script for LinkedIn Company Mining Feature

#import libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time, sys, requests, random
import math
import concurrent.futures
import re

In [2]:
# import and authorize gspread  
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-eaf0e0e671cb.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)
#Global variables
query = ''
queries = []
queriesSplit = []
companies = []
employees = []
locations = []
industries = []
employeeLinks = []
results = []
numberofPages = 0
cellLocationQuery = 'C2'
cellLocationPages = 'B2'
sheetIndexRead = 0
sheetIndexWrite = 2
columnName = 'Company Name (Leave Blank if Using LinkedIn Company Mining)'
columnEmployee = 'Number of Employees'
columnLocation = 'Company Location'
columnIndustry = 'Company Industry'
password = ''
email = 'zach.jl.wong@gmail.com'
drivers = []
namesList = []
maxThreads = 4

#paths/urls
spreadsheet_id = '1vFXonFCyUlEKa1f0s5tvHCKeTek_sAv7rUPYfYss0Qo'
loginurl = 'https://www.linkedin.com/uas/login'
driver_path = '/Users/zacharywong/Documents/Work/Portfolio/DigitalHealthWebscrape/chromedriver'
pathtoLinkedInFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Misc/Companies/Company Web Scrape Tool - Company Names.csv'
pathtoPassword = '/Users/zacharywong/Documents/LinkedIn/LinkedIn-Password-Secret.txt'

#waitTimes
waitLinkedInTime = 10
waitCaptcha = 25
waitSearch= 1
waitLogin = 0.8
waitEmployeeCount=2.5


# xPaths, classes, and javascripts
employeeCountXPATH = "//span[@class='link-without-visited-state t-bold t-black--light']"
employeeCount1XPATH = "//span[@class='org-top-card-secondary-content__see-all t-normal t-black--light link-without-visited-state link-without-hover-state']"
companyIndustryXPATH = "//div[@class='org-top-card-summary-info-list__info-item']"
companyLocationXPATH = "//div[@class='inline-block']"
locationFilterXPATH = "//button[text()='Locations']"
northAmericaXPATH = '//*[@id="companyHqGeo-102221843"]'
submitXPATH = '//*[@id="ember328"]'
searchBarXPath = '//*[@id="global-nav-typeahead"]/input'
companyFilterXPath = '//*[@id="search-reusables__filters-bar"]/ul/li[2]/button'
scrollDownScript = "window.scrollTo(0,document.body.scrollHeight)"
pageButtonClass = "button[type='button']"
companyTitleClass = 'app-aware-link'
companyTitleAttr = 'a'
employeeAttr = 'a'
employeeClass = 'app-aware-link'

In [3]:
def setDrivers(numThreads):
    n = 0
    while n < numThreads:
        service = Service(driver_path)
        driver = webdriver.Chrome(service = service)
        global drivers
        drivers.append(driver)
        n+=1
    #print("drivers: ", drivers)

In [4]:
def splitNames(queries, numThreads):
    spacer = math.floor((len(queries)/numThreads))
    #print("spacer: ", spacer)
    for i in range(0, numThreads, spacer):
        global queriesSplit
        queriesSplit.append(queries[i:i+spacer])
    print("queriesSplit", queriesSplit)
    
    

In [5]:
# helper function to read in value from spreadsheet
def readinValue(cellLocation, sheetIndexRead):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexRead)
    try:
        value = worksheet.acell(cellLocation).value
    except:
        wait()
    return value

In [6]:
def readQueries():
    global queries
    row = 2;
    sheetIndex = 0
    isDone = False
    cellLocationColumn = 'C'
    while (isDone == False):
        cellLocationKeyWords = cellLocationColumn + str(row)
        try:
            keyword = readinValue(cellLocationKeyWords, sheetIndex)
            queries.append(keyword)
            if(keyword == None):
                isDone = True
                break
            else:
                row += 1 
        except:
            wait()
    queries = queries[0:len(queries)-1]
   #print('Queries: ', str(queries))
   

In [7]:
 
# read in query from spreadsheet
def readinQuery(queryIndex):
    global query
    query = queries[queryIndex]
    print('currentQuery: ' + query)
  

In [8]:
  
    
# read in number of pages from spreadsheet
def readinPages():
    global numberofPages
    numberofPages = int(readinValue(cellLocationPages, sheetIndexRead))
    print('numberofPages: ', numberofPages)
    
  

In [9]:

# login to linkedin 
def login(driver):
    driver.get(loginurl)
    emailInput = driver.find_element(By.ID, 'username')
    emailInput.send_keys(email)
    time.sleep(waitLogin)
    passwordInput = driver.find_element(By.ID, 'password')
    with open (pathtoPassword, 'r') as file:
        global password
        password = file.read()
    passwordInput.send_keys(password)
    time.sleep(waitLogin)
    passwordInput.send_keys(Keys.RETURN)

In [10]:
    
# search query on LinkedIn
def searchLinkedIn(driver):
    #search query
    try:
        searchBar = driver.find_element(By.XPATH, searchBarXPath)
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)
    except: 
        time.sleep(waitCaptcha)
        #WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, searchBarXPath)))
        searchBar = driver.find_element(By.XPATH, searchBarXPath)
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)
        
  

In [11]:
      
# Click Company Filter option on LinkedIn
def companyFilter(driver):
    WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, companyFilterXPath)))
    companyButtons = driver.find_elements_by_css_selector(pageButtonClass)
    for button in companyButtons: 
        buttonText = button.text
        print(buttonText)
        if (buttonText == 'Companies'): 
            button.click()
            return;
# Click onto next page 

In [12]:
def locationFilter(driver):
    WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, locationFilterXPATH)))
    locationFilter = driver.find_element(By.XPATH, locationFilterXPATH)
    locationFilter.click()
    print(locationFilter)
    try: 
        WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, northAmericaXPATH)))
        northAmerica = driver.find_element(By.XPATH, northAmericaXPATH)
        submit = driver.find_element(By.XPATH, submitXPATH)
        northAmerica.click()
        submit.click()
    except:
        print("can't find north america filter")
        time.sleep(5)

In [13]:
# Click onto next page 
def nextPage(currentPage, driver):
    nextPageSuccess = False; 
    driver.execute_script(scrollDownScript)
    time.sleep(1)
    #WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ember329"]/button')))
    pages = driver.find_elements_by_css_selector(pageButtonClass)
    for page in pages:
        #time.sleep(1)
        buttonNumber = page.text
        #print(buttonNumber) 
        if (buttonNumber == str(currentPage)):
            #print('hello')
            page.click()
            nextPageSuccess = True; 
            break
    if (nextPageSuccess == False):
        for page in pages:
            buttonNumber = page.text
            if (buttonNumber == '…'):
                #print("click more option")
                page.click()
    

In [14]:
# Click onto next page 
def nextPage(currentPage, driver):
    nextPageSuccess = False; 
    driver.execute_script(scrollDownScript)
    time.sleep(1)
    #WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ember329"]/button')))
    pages = driver.find_elements_by_css_selector(pageButtonClass)
    for page in pages:
        #time.sleep(1)
        buttonNumber = page.text
        #print(buttonNumber) 
        if (buttonNumber == str(currentPage)):
            #print('hello')
            page.click()
            nextPageSuccess = True; 
            break
    if (nextPageSuccess == False):
        for page in pages:
            buttonNumber = page.text
            if (buttonNumber == '…'):
                #print("click more option")
                page.click()
                return 

In [15]:
# add companies into a pandas dataframe 
def addCompanies(driver):
    global companies
    currentPage = 1
    companyName = ''
    pattern = re.compile('^https://www.linkedin.com/company/')
    while (currentPage <= numberofPages):
        time.sleep(2)
        soup = bs(driver.page_source, 'html.parser')
        companySearch = soup.find_all(companyTitleAttr, class_ = companyTitleClass)
        #print("companySearch: ", companySearch)
        try: 
            employeeSearch = soup.find_all('a', class_ = companyTitleClass, href = True)
            #print(employeeSearch)
        except:
            print("can't find links")
        for company in companySearch:
            companyName = company.text
            if '\n\n\n' not in companyName and companyName not in companies:
                companies.append(companyName.replace('\n', ''))
        for employee in employeeSearch:
            try: 
                global employeeLinks
                employeeLink = employee.get('href')
                if pattern.match(employeeLink) and employeeLink not in employeeLinks:
                    print("employeeLink: ", employeeLink)
                    employeeLinks.append(employeeLink)                    
            except: 
                print("failed inputting employees")
        currentPage += 1
        nextPage(currentPage, driver)
    return companies

In [16]:
def addIndustry(driver):
    try:
        WebDriverWait(driver, waitEmployeeCount).until(EC.presence_of_element_located((By.XPATH, companyIndustryXPATH)))
        industry = driver.find_element(By.XPATH, companyIndustryXPATH)
        print(industry.text)
        global industries
        industries.append(industry.text)
    except:
        print("cannot find industry")
        industries.append(None)
    return
    

In [17]:
def addLocation(driver):
    pattern = '[0-9]+'
    try:
        WebDriverWait(driver, waitEmployeeCount).until(EC.presence_of_element_located((By.XPATH, companyLocationXPATH)))
        location = driver.find_element(By.XPATH, companyLocationXPATH)
    except:
        print("cannot find location")
        global locations
        locations.append(None)
        return
    location = location.text
    location = re.split(pattern, location)
    location = location[0]
    print(location)
    locations.append(location)
    return
    

In [18]:
def addSecondaryInfo():
    print('employee links: ', employeeLinks)
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    login(driver)
    time.sleep(waitCaptcha)
    count = 0
    for link in employeeLinks:
        print('count: ', count)
        count += 1
        isDone = False
        driver.get(link)
        try:
            WebDriverWait(driver, waitEmployeeCount).until(EC.presence_of_element_located((By.XPATH, employeeCountXPATH)))
            employee = driver.find_element(By.XPATH, employeeCountXPATH)
            print(employee.text)
        except: 
            print("cannot find regular employee")
            try:
                WebDriverWait(driver, waitEmployeeCount).until(EC.presence_of_element_located((By.XPATH, employeeCount1XPATH)))
                employee = driver.find_element(By.XPATH, employeeCount1XPATH)
                print(employee.text)
            except:
                print("cannot find employee location")
                global employees 
                employees.append(None)
                isDone = True
       
        addIndustry(driver)
        addLocation(driver)      
        
        if isDone == False: 
            employeeCount = employee.text.split(' ')
            #primary case
            if (employeeCount[1] == 'all'):
                employeeCount = employeeCount[2]
            #secondary case
            elif(employeeCount[1] == 'employees'):
                employeeCount = employeeCount[0]
            #1 employee for primary case 
            else:
                employeeCount = employeeCount[1]
            print("count: " + employeeCount + ' employeeLink: ' + str(link))
            employees.append(employeeCount)
        else: 
            continue
    print('employeeCounts: ' + str(employees) + str(len(employees)) + '\n' )
    print('locations: ' + str(locations) + str(len(locations)) + '\n')
    print('industries: ' + str(industries) + str(len(industries)) + '\n')
    time.sleep(0.4)


In [19]:
def makeDF():
    tuples = list(zip(companies, employees, locations, industries))
    df = pd.DataFrame(tuples, columns = [columnName, columnEmployee, columnLocation, columnIndustry])
    return df 

In [20]:
# export final list of companies to CSV 
def exportCSV(df):
    df.to_csv(pathtoLinkedInFile)

In [21]:
# Update spreadsheet with list of companies
def updateSpreadSheet(df, sheetIndexWrite):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexWrite)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    

In [22]:
  
# run all functions to scrape LinkedIn companies  
#def runLinkedIn(driver, query):
    #service = Service(driver_path)
    #driver = webdriver.Chrome(service = service)
    #readinQuery(queryIndex)
 #   readinPages()
 #   login(driver)
 #   searchLinkedIn(driver, query)
 #   companyFilter(driver)
 #   locationFilter(driver)
#    companies = addCompanies(driver)
    
# run all functions to scrape LinkedIn companies  
def runLinkedIn(queryIndex):
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    readinQuery(queryIndex)
    readinPages()
    login(driver)
    searchLinkedIn(driver)
    companyFilter(driver)
    locationFilter(driver)
    companies = addCompanies(driver)
    

In [23]:
def runQueries(driver, query):
    runLinkedIn(driver, query)

In [24]:
#run program 
#startTime = time.time()
#readQueries()
#print(queries)
#numThreads = min(maxThreads, len(queries))
#setDrivers(numThreads)
#splitNames(queries, numThreads)
#print("queries: ", queries)
#service = Service(driver_path)
#driver = webdriver.Chrome(service = service)
#for query in queries:
#    runQueries(driver, query)
#with concurrent.futures.ThreadPoolExecutor(max_workers=numThreads) as executor:
#    executor.map(runQueries, drivers, queries)
#print(employeeLinks)
#addSecondaryInfo()
#df = makeDF()
#exportCSV(df)
#updateSpreadSheet(df, sheetIndexWrite)
#endTime = time.time()
#elapsedTime = endTime - startTime
#print('Time Elapsed: ', elapsedTime)
#df


# run program 
startTime = time.time()
readQueries()
print('Queries: ', queries)
queryIndex = 0
while queryIndex < len(queries):
    try:
        runLinkedIn(queryIndex)
    except:
        print("can't run query")
    print(companies)
    queryIndex += 1
addSecondaryInfo()
df = makeDF()
exportCSV(df)
updateSpreadSheet(df, sheetIndexWrite)
endTime = time.time()
elapsedTime = endTime - startTime
print('Time Elapsed: ', elapsedTime)
df

Queries:  ['personal finance education', 'finance certification education', 'accounting certification education']
currentQuery: personal finance education
numberofPages:  10


/var/folders/gl/jgk7f2w528qgjggck6glqp200000gn/T/ipykernel_25316/1096647697.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  companyButtons = driver.find_elements_by_css_selector(pageButtonClass)



Me
Work
People
Companies
<selenium.webdriver.remote.webelement.WebElement (session="0f670a78575bf06af66165baff0d72b5", element="b7f3bdb6-20cf-4329-aaa4-2963c995483b")>
can't find north america filter
employeeLink:  https://www.linkedin.com/company/education-at-work/
employeeLink:  https://www.linkedin.com/company/personal-finance-education-services-inc/
employeeLink:  https://www.linkedin.com/company/moneycheck-personal-finance-education/
employeeLink:  https://www.linkedin.com/company/personal-finance-employee-education-foundation/
employeeLink:  https://www.linkedin.com/company/ramseysolutions-lampo/
employeeLink:  https://www.linkedin.com/company/finance-student-association/
employeeLink:  https://www.linkedin.com/company/non-profitpersonalcareerguidance&development/
employeeLink:  https://www.linkedin.com/company/personal-money-store/
employeeLink:  https://www.linkedin.com/company/personal-finance-association-pfa-/
employeeLink:  https://www.linkedin.com/company/personal-finance-

/var/folders/gl/jgk7f2w528qgjggck6glqp200000gn/T/ipykernel_25316/943768588.py:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  pages = driver.find_elements_by_css_selector(pageButtonClass)


employeeLink:  https://www.linkedin.com/company/personal-finance-boss/
employeeLink:  https://www.linkedin.com/company/stanford-graduate-school-of-business-executive-education/
employeeLink:  https://www.linkedin.com/company/fisherpersonalfinance/
employeeLink:  https://www.linkedin.com/company/councileconed/
employeeLink:  https://www.linkedin.com/company/center-for-global-development/
employeeLink:  https://www.linkedin.com/company/investopedia-ulc/
employeeLink:  https://www.linkedin.com/company/rsf-social-finance/
employeeLink:  https://www.linkedin.com/company/the-university-of-tampa/
employeeLink:  https://www.linkedin.com/company/isenberg-school-of-management-umass-amherst/
employeeLink:  https://www.linkedin.com/company/eastern-connecticut-state-university/
employeeLink:  https://www.linkedin.com/company/postuniversity/
employeeLink:  https://www.linkedin.com/company/north-central-college/
employeeLink:  https://www.linkedin.com/company/perennial-resources-international/
employ

...

See 1 employee on LinkedIn
Financial Services
Piscataway, NJ 
count: 1 employeeLink: https://www.linkedin.com/company/financial-statistics-and-risk-management-program/
count:  158
cannot find regular employee
cannot find employee location
Higher Education

count:  159
See all 2,707 employees on LinkedIn
McKinney, TX
McKinney, TX 
count: 2,707 employeeLink: https://www.linkedin.com/company/6688/
count:  160
See all 5 employees on LinkedIn
Higher Education
Toronto, Ontario 
count: 5 employeeLink: https://www.linkedin.com/company/rotman-business-analytics-club/
count:  161
See all 10 employees on LinkedIn
Higher Education
San Diego, CA 
count: 10 employeeLink: https://www.linkedin.com/company/institute-of-business-&-finance/
count:  162
See all 7 employees on LinkedIn
Renewables & Environment
Sherwood Park, Alberta 
count: 7 employeeLink: https://www.linkedin.com/company/3d-energy-ltd-/
count:  163
See all 11 employees on LinkedIn
Financial Services
Cupertino, CA 
count: 11 employeeLink: 

cannot find regular employee
cannot find employee location
Higher Education
Beijing, Beijing 
count:  213
cannot find regular employee
cannot find employee location
Accounting
Victoria, British Columbia 
count:  214
cannot find regular employee
cannot find employee location
Real Estate
San Diego, CA 
count:  215
cannot find regular employee
cannot find employee location
Professional Training & Coaching
Chesterfield, VA 
count:  216
cannot find regular employee
cannot find employee location
Information Services
Colorado Springs , Colorado 
count:  217
cannot find regular employee
cannot find employee location
Doral, Florida
Doral, Florida 
count:  218
cannot find regular employee
cannot find employee location
Higher Education
Flint, MI 
count:  219
See all 29 employees on LinkedIn
Management Consulting
Arlington, VA 
count: 29 employeeLink: https://www.linkedin.com/company/imn-solutions/
count:  220
See all 39 employees on LinkedIn
Brooklyn, New York
Brooklyn, New York 
count: 39 employ

cannot find regular employee
cannot find employee location
E-learning
Washington, DC 
count:  267
See all 4 employees on LinkedIn
Accounting
Frederick, MD 
count: 4 employeeLink: https://www.linkedin.com/company/almacy-co-cpas/
count:  268
See all 3 employees on LinkedIn
Writing & Editing
Brooklyn, NY 
count: 3 employeeLink: https://www.linkedin.com/company/wordsworth-associates-inc./
count:  269
cannot find regular employee
cannot find employee location
Accounting
Madison, Alabama 
count:  270
cannot find regular employee
cannot find employee location
Management Consulting
Stone Mountain, Georgia 
count:  271
See 1 employee on LinkedIn
E-learning
Philadelphia, Pa 
count: 1 employeeLink: https://www.linkedin.com/company/cpe-university-llc/
count:  272
See all 3 employees on LinkedIn
Events Services
Denver, CO 
count: 3 employeeLink: https://www.linkedin.com/company/oes-management-llc/
count:  273
cannot find regular employee
cannot find employee location
Professional Training & Coachin

Time Elapsed:  1074.2274918556213


,Company Name (Leave Blank if Using LinkedIn Company Mining),Number of Employees,Company Location,Company Industry
0,Education at Work,880,"Tempe, Arizona",Outsourcing/Offshoring
1,PERSONAL FINANCE EDUCATION SERVICES INC,2,"Ann Arbor, Michigan",Individual & Family Services
2,MoneyCheck: Personal Finance Education,None,"Atlanta, GA",Financial Services
3,Personal Finance Employee Education Fund,3,"Lombard, Illinois",Financial Services
4,Ramsey Solutions,"1,321","Franklin, TN",Financial Services
...,...,...,...,...
278,Stella Sustainability,1,"Alameda, California",Computer Software
279,Central Illinos Chapter of the Institute of Ma...,1,"Tucson, AZ",Accounting
280,"ManageWise, Inc.",2,"San Francisco, California",Financial Services
281,CollegeOne,None,"Barrington, NH",Information Services
